### Import libraries

In [37]:
import tensorflow as tf
import numpy as np
import os
import random
import splitfolders

tfk = tf.keras
tfkl = tf.keras.layers
print(tf.__version__)

2.10.0


### Set seed for reproducibility

In [38]:
# Random seed for reproducibility
seed = 41

random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

### Suppress warnings

In [39]:
import warnings
import logging

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=Warning)
tf.get_logger().setLevel('INFO')
tf.autograph.set_verbosity(0)

tf.get_logger().setLevel(logging.ERROR)
tf.get_logger().setLevel('ERROR')
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

In [40]:
cd data

[Errno 2] No such file or directory: 'data'
/Users/nicolacecere/GitHub/Artificial-Neural-Networks-and-Deep-Learning


In [41]:
!ls

Exception-tl-ft-Nic.ipynb             SubmissionPreTrain
Exception_Best_Score.ipynb            SubmissionPreTrain copy
FineTuningModel                       SubmissionPreTrain copy 2.zip
ModelWithAugNic.ipynb                 SubmissionPreTrain copy 3.zip
Model_Federico.ipynb                  SubmissionPreTrain copy 4.zip
Model_Nicola.ipynb                    SubmissionPreTrain copy.zip
Model_without_data_augmentation.ipynb TransferLearningModel
Pre_trained_DenseNet.ipynb            Vgg16-tl-ft-Nic.ipynb
Pre_trained_EfficentNet.ipynb         data_augmentation_experiments
README.md                             model.png
SplitData                             requirements.txt
SubmissionCnnCustom                   training_data_final


In [14]:
splitfolders.ratio('/Users/nicolacecere/GitHub/Artificial-Neural-Networks-and-Deep-Learning/SplitData/data/training_data_final/'
, output='output1', seed=seed, ratio=(0.8, 0.2))

In [42]:
# Download the data
training = '/Users/nicolacecere/GitHub/Artificial-Neural-Networks-and-Deep-Learning/SplitData/data/output/train'
validation = '/Users/nicolacecere/GitHub/Artificial-Neural-Networks-and-Deep-Learning/SplitData/data/output/val'
test = '/Users/nicolacecere/GitHub/Artificial-Neural-Networks-and-Deep-Learning/SplitData/data/output/test'

In [22]:
#no test
training = '/Users/nicolacecere/GitHub/Artificial-Neural-Networks-and-Deep-Learning/SplitData/data/output1/train'
validation = '/Users/nicolacecere/GitHub/Artificial-Neural-Networks-and-Deep-Learning/SplitData/data/output1/val'

### Process data

In [43]:
# Images are divided into folders, one for each class.
# If the images are organized in such a way, we can exploit the
# ImageDataGenerator to read them from disk.
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.efficientnet_v2 import preprocess_input

# Create an instance of ImageDataGenerator for training, validation, and test sets
train_data_gen = ImageDataGenerator(preprocessing_function = preprocess_input)
valid_data_gen = ImageDataGenerator(preprocessing_function = preprocess_input)
test_data_gen = ImageDataGenerator(preprocessing_function = preprocess_input)

# Obtain a data generator with the 'ImageDataGenerator.flow_from_directory' method
train_gen = train_data_gen.flow_from_directory(directory=training,
                                               target_size=(96,96),
                                               color_mode='rgb',
                                               classes=None, # can be set to labels
                                               class_mode='categorical',
                                               batch_size=32,
                                               shuffle=True,
                                               seed=seed
                                               )
valid_gen = train_data_gen.flow_from_directory(directory=validation,
                                               target_size=(96,96),
                                               color_mode='rgb',
                                               classes=None, # can be set to labels
                                               class_mode='categorical',
                                               batch_size=32,
                                               shuffle=True,
                                               seed=seed
                                               )
test_gen = train_data_gen.flow_from_directory(directory=test,
                                              target_size=(96,96),
                                              color_mode='rgb',
                                              classes=None, # can be set to labels
                                              class_mode='categorical',
                                              batch_size=32,
                                              shuffle=False,
                                              seed=seed
                                              )

Found 2829 images belonging to 8 classes.
Found 351 images belonging to 8 classes.
Found 362 images belonging to 8 classes.


In [44]:
# Create an instance of ImageDataGenerator with Data Augmentation
aug_train_data_gen = ImageDataGenerator(rotation_range=20,
                                        horizontal_flip= True,
                                        vertical_flip= True,
                                        brightness_range=(0.6,1.4),
                                        zoom_range=0.6,
                                        fill_mode='nearest',
                                        preprocessing_function = preprocess_input) # rescale value is multiplied to the image

# Obtain a data generator with the 'ImageDataGenerator.flow_from_directory' method
aug_train_gen = aug_train_data_gen.flow_from_directory(directory=training,
                                                       target_size=(96,96),
                                                       color_mode='rgb',
                                                       classes=None, # can be set to labels
                                                       class_mode='categorical',
                                                       batch_size=32,
                                                       shuffle=True,
                                                       seed=seed)

Found 2829 images belonging to 8 classes.


### Models metadata

In [45]:
input_shape = (96, 96, 3)
batch_size = 32
epochs = 200

### Standard model

### Transfer learning

In [46]:
# Download and plot the VGG16 model
supernet = tfk.applications.EfficientNetV2B3(
    include_top=False,
    weights="imagenet",
    input_shape=(96,96,3)
)
supernet.summary()

Model: "efficientnetv2-b3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 96, 96, 3)]  0           []                               
                                                                                                  
 rescaling_2 (Rescaling)        (None, 96, 96, 3)    0           ['input_5[0][0]']                
                                                                                                  
 normalization_2 (Normalization  (None, 96, 96, 3)   0           ['rescaling_2[0][0]']            
 )                                                                                                
                                                                                                  
 stem_conv (Conv2D)             (None, 48, 48, 40)   1080        ['normalization_2

In [47]:
def scheduler(epoch, lr):
   if epoch < 10:
     return lr
   else:
     return lr * tf.math.exp(-0.1)

In [48]:
from datetime import datetime

def create_folders_and_callbacks(model_name):

  exps_dir = os.path.join('data_augmentation_experiments')
  if not os.path.exists(exps_dir):
      os.makedirs(exps_dir)

  now = datetime.now().strftime('%b%d_%H-%M-%S')

  exp_dir = os.path.join(exps_dir, model_name + '_' + str(now))
  if not os.path.exists(exp_dir):
      os.makedirs(exp_dir)

  callbacks = []

  # Model checkpoint
  # ----------------
  ckpt_dir = os.path.join(exp_dir, 'ckpts')
  if not os.path.exists(ckpt_dir):
      os.makedirs(ckpt_dir)

  ckpt_callback = tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(ckpt_dir, 'cp.ckpt'),
                                                     save_weights_only=True, # True to save only weights
                                                     save_best_only=False) # True to save only the best epoch
  callbacks.append(ckpt_callback)

  # Visualize Learning on Tensorboard
  # ---------------------------------
  tb_dir = os.path.join(exp_dir, 'tb_logs')
  if not os.path.exists(tb_dir):
      os.makedirs(tb_dir)

  # By default shows losses and metrics for both training and validation
  tb_callback = tf.keras.callbacks.TensorBoard(log_dir=tb_dir,
                                               profile_batch=0,
                                               histogram_freq=1)  # if > 0 (epochs) shows weights histograms
  callbacks.append(tb_callback)

  # Early Stopping
  # --------------
  es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True)
  callbacks.append(es_callback)


  LRS_callback = tf.keras.callbacks.LearningRateScheduler(scheduler)
  callbacks.append(LRS_callback)

  return callbacks

In [49]:
# Use the supernet as feature extractor
supernet.trainable = False

inputs = tfk.Input(shape=(96,96,3))
x = supernet(inputs)
x = tfkl.Flatten(name='Flattening')(x)
x = tfkl.Dropout(0.2, seed=seed)(x)
x = tfkl.Dense(
    128, 
    #activation='relu',
    kernel_initializer = tfk.initializers.HeUniform(seed))(x)
leaky_relu_layer = tfkl.LeakyReLU()(x)
x = tfkl.Dropout(0.2, seed=seed)(leaky_relu_layer)

outputs = tfkl.Dense(
    8,
    activation='softmax',
    kernel_initializer = tfk.initializers.GlorotUniform(seed))(x)


# Connect input and output through the Model class
tl_model = tfk.Model(inputs=inputs, outputs=outputs, name='model')

# Compile the model
tl_model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(), metrics='accuracy')
tl_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 96, 96, 3)]       0         
                                                                 
 efficientnetv2-b3 (Function  (None, 3, 3, 1536)       12930622  
 al)                                                             
                                                                 
 Flattening (Flatten)        (None, 13824)             0         
                                                                 
 dropout_4 (Dropout)         (None, 13824)             0         
                                                                 
 dense_4 (Dense)             (None, 128)               1769600   
                                                                 
 leaky_re_lu_2 (LeakyReLU)   (None, 128)               0         
                                                             

In [50]:
# Train the model
aug_callbacks = create_folders_and_callbacks(model_name='CNN_Aug_Pre')

tl_history = tl_model.fit(
    x = aug_train_gen,
    batch_size = 32,
    epochs = 200,
    validation_data = valid_gen,
    callbacks = aug_callbacks
).history

Epoch 1/200
89/89 [==============================] - 76s 631ms/step - loss: 2.1546 - accuracy: 0.3772 - val_loss: 1.2751 - val_accuracy: 0.5470 - lr: 0.0010
Epoch 2/200
89/89 [==============================] - 27s 307ms/step - loss: 1.6673 - accuracy: 0.4687 - val_loss: 1.1625 - val_accuracy: 0.5954 - lr: 0.0010
Epoch 3/200
89/89 [==============================] - 25s 285ms/step - loss: 1.5910 - accuracy: 0.5019 - val_loss: 1.2068 - val_accuracy: 0.5613 - lr: 0.0010
Epoch 4/200
89/89 [==============================] - 25s 276ms/step - loss: 1.5015 - accuracy: 0.5143 - val_loss: 1.1493 - val_accuracy: 0.6239 - lr: 0.0010
Epoch 5/200
89/89 [==============================] - 25s 282ms/step - loss: 1.4597 - accuracy: 0.5313 - val_loss: 1.2027 - val_accuracy: 0.6125 - lr: 0.0010
Epoch 6/200
89/89 [==============================] - 25s 282ms/step - loss: 1.3268 - accuracy: 0.5539 - val_loss: 1.0125 - val_accuracy: 0.6154 - lr: 0.0010
Epoch 7/200
89/89 [==============================] - 28s 3

In [53]:
# Save the best model

tl_model.save('TransferLearningModel_LearningRate')


In [54]:
model_aug = tfk.models.load_model("TransferLearningModel_LearningRate")
model_aug_test_metrics = model_aug.evaluate(test_gen, return_dict=True)

print()
print("Test metrics with VGG")
print(model_aug_test_metrics)

12/12 [==============================] - 20s 1s/step - loss: 0.9414 - accuracy: 0.6602

Test metrics with VGG
{'loss': 0.9414159655570984, 'accuracy': 0.6602209806442261}


### Fine tuning

In [55]:
# Re-load the model after transfer learning
ft_model = tfk.models.load_model('TransferLearningModel_LearningRate')
ft_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 96, 96, 3)]       0         
                                                                 
 efficientnetv2-b3 (Function  (None, 3, 3, 1536)       12930622  
 al)                                                             
                                                                 
 Flattening (Flatten)        (None, 13824)             0         
                                                                 
 dropout_4 (Dropout)         (None, 13824)             0         
                                                                 
 dense_4 (Dense)             (None, 128)               1769600   
                                                                 
 leaky_re_lu_2 (LeakyReLU)   (None, 128)               0         
                                                             

In [56]:
# Set all VGG layers to True
ft_model.get_layer('efficientnetv2-b3').trainable = True
for i, layer in enumerate(ft_model.get_layer('efficientnetv2-b3').layers):
   print(i, layer.name, layer.trainable)

0 input_5 True
1 rescaling_2 True
2 normalization_2 True
3 stem_conv True
4 stem_bn True
5 stem_activation True
6 block1a_project_conv True
7 block1a_project_bn True
8 block1a_project_activation True
9 block1b_project_conv True
10 block1b_project_bn True
11 block1b_project_activation True
12 block1b_drop True
13 block1b_add True
14 block2a_expand_conv True
15 block2a_expand_bn True
16 block2a_expand_activation True
17 block2a_project_conv True
18 block2a_project_bn True
19 block2b_expand_conv True
20 block2b_expand_bn True
21 block2b_expand_activation True
22 block2b_project_conv True
23 block2b_project_bn True
24 block2b_drop True
25 block2b_add True
26 block2c_expand_conv True
27 block2c_expand_bn True
28 block2c_expand_activation True
29 block2c_project_conv True
30 block2c_project_bn True
31 block2c_drop True
32 block2c_add True
33 block3a_expand_conv True
34 block3a_expand_bn True
35 block3a_expand_activation True
36 block3a_project_conv True
37 block3a_project_bn True
38 block3b_

In [57]:
# Freeze first N layers, e.g., until 14th
for i, layer in enumerate(ft_model.get_layer('efficientnetv2-b3').layers[:190]):
  layer.trainable=False
for i, layer in enumerate(ft_model.get_layer('efficientnetv2-b3').layers):
   print(i, layer.name, layer.trainable)
ft_model.summary()

0 input_5 False
1 rescaling_2 False
2 normalization_2 False
3 stem_conv False
4 stem_bn False
5 stem_activation False
6 block1a_project_conv False
7 block1a_project_bn False
8 block1a_project_activation False
9 block1b_project_conv False
10 block1b_project_bn False
11 block1b_project_activation False
12 block1b_drop False
13 block1b_add False
14 block2a_expand_conv False
15 block2a_expand_bn False
16 block2a_expand_activation False
17 block2a_project_conv False
18 block2a_project_bn False
19 block2b_expand_conv False
20 block2b_expand_bn False
21 block2b_expand_activation False
22 block2b_project_conv False
23 block2b_project_bn False
24 block2b_drop False
25 block2b_add False
26 block2c_expand_conv False
27 block2c_expand_bn False
28 block2c_expand_activation False
29 block2c_project_conv False
30 block2c_project_bn False
31 block2c_drop False
32 block2c_add False
33 block3a_expand_conv False
34 block3a_expand_bn False
35 block3a_expand_activation False
36 block3a_project_conv False
3

In [58]:
# Compile the model
ft_model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(), metrics='accuracy')

In [59]:
# Fine-tune the model
ft_history = ft_model.fit(
    x = aug_train_gen,
    batch_size = 32,
    epochs = 200,
    validation_data = valid_gen,
    callbacks = aug_callbacks
).history

Epoch 1/200
89/89 [==============================] - 152s 1s/step - loss: 1.6789 - accuracy: 0.4733 - val_loss: 0.8767 - val_accuracy: 0.6724 - lr: 0.0010
Epoch 2/200
89/89 [==============================] - 51s 568ms/step - loss: 1.2086 - accuracy: 0.5794 - val_loss: 0.7342 - val_accuracy: 0.7464 - lr: 0.0010
Epoch 3/200
89/89 [==============================] - 44s 497ms/step - loss: 1.0587 - accuracy: 0.6246 - val_loss: 0.6865 - val_accuracy: 0.7607 - lr: 0.0010
Epoch 4/200
89/89 [==============================] - 44s 493ms/step - loss: 0.9864 - accuracy: 0.6543 - val_loss: 0.7184 - val_accuracy: 0.7692 - lr: 0.0010
Epoch 5/200
89/89 [==============================] - 40s 453ms/step - loss: 0.9184 - accuracy: 0.6914 - val_loss: 0.6142 - val_accuracy: 0.7949 - lr: 0.0010
Epoch 6/200
89/89 [==============================] - 40s 447ms/step - loss: 0.8562 - accuracy: 0.7052 - val_loss: 0.6422 - val_accuracy: 0.7892 - lr: 0.0010
Epoch 7/200
89/89 [==============================] - 41s 458

In [60]:
ft_model.save('FineTuningModel_LearningRate')
del ft_model

In [61]:
model_aug = tfk.models.load_model("FineTuningModel_LearningRate")
model_aug_test_metrics = model_aug.evaluate(test_gen, return_dict=True)

print()
print("Test metrics with VGG")
print(model_aug_test_metrics)

12/12 [==============================] - 26s 1s/step - loss: 0.4521 - accuracy: 0.8619

Test metrics with VGG
{'loss': 0.45210230350494385, 'accuracy': 0.8618783950805664}
